In [1]:
push!(LOAD_PATH, "../src")
using Quaternions
using FixedSizeArrays
using RigidBodyDynamics

In [2]:
mechanism = Mechanism{Float64}("world")
world = root_body(mechanism)

RigidBody: "world"

In [3]:
body1 = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body1")));
joint1 = Joint("joint1", QuaternionFloating());
joint1ToWorld = rand(Transform3D{Float64}, joint1.frameBefore, world.frame)
body1ToJoint1 = rand(Transform3D{Float64}, body1.frame, joint1.frameAfter)
attach!(mechanism, world, joint1, joint1ToWorld, body1, body1ToJoint1);

In [4]:
joint2 = Joint("joint2", Revolute(rand(Vec{3, Float64})));
joint2ToBody1 = rand(Transform3D{Float64}, joint2.frameBefore, body1.frame)
body2frame = CartesianFrame3D("body2")
body2 = RigidBody(rand(SpatialInertia{Float64}, body2frame));
body2ToJoint2 = rand(Transform3D{Float64}, body2frame, joint2.frameAfter)
attach!(mechanism, body1, joint2, joint2ToBody1, body2, body2ToJoint2);

In [5]:
mechanism

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2

In [6]:
x = MechanismState(Float64, mechanism);

In [7]:
rand!(x)
setdirty!(x)
configuration(x, joint1)

7-element Array{Float64,1}:
 -0.00633857
 -0.195537  
 -0.0906361 
  0.976479  
  0.412931  
  0.502242  
  0.7606    

In [8]:
num_positions(x)

8

In [9]:
num_velocities(x)

7

In [10]:
relative_transform(x, joint2.frameAfter, body1.frame)

Transform3D from "after_joint2" to "body1":
rotation: 1.4892190107226417 rad about [0.7900154047116512,-0.45249392524269544,0.41367246456193346], translation: Vec(0.08443597430394956,0.43553700925428784,0.9306881711031733)


In [11]:
center_of_mass(x)

Point3D in "world": Vec(1.5920897510105874,-0.07711028695362969,0.8038335809017787)

In [12]:
mass(mechanism)

0.520131422420598

In [13]:
transform(x, Point3D{Float64}(joint2.frameAfter), root_frame(mechanism))

Point3D in "world": Vec(2.005808070445764,1.068884452750909,1.0991547645996995)

In [14]:
relative_twist(x, body2, body1)

Twist of "after_joint2" w.r.t "after_joint1" in "world":
angular: Vec(0.0736017057171634,-0.04540268447625062,-0.016300244096506322), linear: Vec(0.03248149947688728,0.1135948266806934,-0.16974078987938976)

In [15]:
relative_twist(x, body1, world) + relative_twist(x, body2, body1) + relative_twist(x, world, body2)

Twist of "world" w.r.t "world" in "world":
angular: Vec(0.0,0.0,0.0), linear: Vec(0.0,0.0,0.0)

In [16]:
p = path(mechanism, body2, world)
J = geometric_jacobian(x, p)

GeometricJacobian: body: "world", base: "after_joint2", expressed in "world":
6x7 Array{Float64,2}:
 -0.71408   -0.751795  -0.640747   …  -0.0       -0.0        -0.0       
  0.440495   0.123743   0.0948633     -0.0       -0.0        -0.0       
  0.158144  -0.647682   0.761869      -0.0       -0.0        -0.0       
 -0.315134  -0.501299   0.435583      -0.751795  -0.640747   -0.155717  
 -1.10209   -0.461342  -0.434801       0.123743   0.0948633  -0.987769  
  1.64682    0.493739   0.420473   …  -0.647682   0.761869   -0.00797047

In [17]:
Twist(J, velocity_vector(x, p))

Twist of "world" w.r.t "after_joint2" in "world":
angular: Vec(-0.27364569895269086,-0.27950408036050445,0.0072817236393081415), linear: Vec(-0.5284219833396636,-0.13205477827976816,-0.20543347672506374)

In [18]:
mass_matrix(x)

7x7 Symmetric{Float64,Array{Float64,2}}:
  0.913352     -0.685585      0.451194     …   0.491447      0.369112 
 -0.685585      1.53429       0.237588        -0.711951      0.235893 
  0.451194      0.237588      1.64441         -1.05168e-17   0.872931 
  6.93889e-18  -0.247297     -0.491447         1.5786e-16   -0.392078 
  0.247297      6.93889e-18   0.711951         6.07153e-17   0.346073 
  0.491447     -0.711951     -1.05168e-17  …   0.520131      0.0701346
  0.369112      0.235893      0.872931         0.0701346     0.692429 

In [19]:
A = momentum_matrix(x)

MomentumMatrix expressed in "world":
6x7 Array{Float64,2}:
 -0.0986739   0.827738    0.191027    …   0.0702189  -0.413305     0.173245 
  0.554141   -0.0109938   1.62876         0.898796    0.058505     0.770774 
  0.979014   -1.43901     0.0676162      -0.0528571   0.824213     0.109453 
  0.234982   -0.296779    0.0867135       0.333273    0.0809935   -0.0620959
  0.461977   -0.672643   -0.00672474     -0.0493414   0.51377      0.0849642
 -0.18449    -0.165844   -0.860715    …  -0.396272    0.00414569  -0.517045 

In [20]:
Momentum(A, velocity_vector(x))

Momentum expressed in "world":
angular: Vec(0.17448361229356257,0.5981935120059563,-0.044167091329264926), linear: Vec(0.3877588190914379,-0.05781097936178742,-0.13523021300859053)

In [21]:
accel = SpatialAcceleration(joint2.frameAfter, world.frame, joint2.frameAfter, rand(Vec{3, Float64}), rand(Vec{3, Float64}))
transform(x, accel, world.frame)

SpatialAcceleration of "after_joint2" w.r.t "world" in "world":
angular: Vec(0.6912372787738521,-0.6385226397687234,-0.3856286844863212), linear: Vec(0.15953372745201186,0.5646268787725857,-2.265473650910321)

In [22]:
bias_acceleration(x, body2)

SpatialAcceleration of "after_joint2" w.r.t "world" in "world":
angular: Vec(-0.0048865945366845165,0.003924544408473864,-0.032996226397272344), linear: Vec(-0.039441371035623535,0.06994597739751483,-0.0117052094049475)

In [23]:
v̇ = rand(num_velocities(mechanism))
τ = inverse_dynamics(x, v̇)

7-element Array{Float64,1}:
 -0.800173
 -0.988999
 -6.23117 
  2.75224 
 -2.82127 
  0.160491
 -3.53819 

In [24]:
relative_acceleration(x, body1, body2, v̇)

SpatialAcceleration of "after_joint1" w.r.t "after_joint2" in "world":
angular: Vec(-0.6256416739351145,0.3850294308810687,0.17263653208982233), linear: Vec(-0.2388198923817201,-1.0430857800207214,1.4658340952621312)

In [25]:
rand_chain_mechanism(Float64, QuaternionFloating, Revolute{Float64}, Revolute{Float64})

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2
      Vertex: body3, Edge: joint3

In [26]:
rand_tree_mechanism(Float64, [QuaternionFloating; [Revolute{Float64} for i = 1 : 10]]...)

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2
      Vertex: body4, Edge: joint4
        Vertex: body6, Edge: joint6
          Vertex: body7, Edge: joint7
        Vertex: body10, Edge: joint10
    Vertex: body5, Edge: joint5
    Vertex: body11, Edge: joint11
  Vertex: body3, Edge: joint3
  Vertex: body8, Edge: joint8
  Vertex: body9, Edge: joint9